## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Описание данных
- _children_ — количество детей в семье
- _days_employed_ — общий трудовой стаж в днях
- _dob_years_ — возраст клиента в годах
- _education_ — уровень образования клиента
- _education_id_ — идентификатор уровня образования
- _family_status_ — семейное положение
- _family_status_id_ — идентификатор семейного положения
- _gender_ — пол клиента
- _income_type_ — тип занятости
- _debt_ — имел ли задолженность по возврату кредитов
- _total_income_ — ежемесячный доход
- _purpose_ — цель получения кредита

## Содержание

1. [Шаг 1. Откроем файл с данными и изучим общую информацию](#Шаг1)
2. [Шаг 2. Предобработка данных](#Шаг2)
3. [Шаг 3. Ответы на вопросы](#Шаг3)
 1. [3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок?](#Шаг31)
 2. [3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?](#Шаг32)
 3. [3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?](#Шаг33)
 4. [3.4 Как разные цели кредита влияют на его возврат в срок?](#Шаг34)
4. [Шаг 4. Общий вывод](#Шаг4)

### Шаг 1. Откроем файл с данными и изучим общую информацию <a name=Шаг1> </a>

In [61]:
import pandas as pd
import os
os.getcwd()
paying_capacity = pd.read_csv(os.path.join(os.getcwd(), 'Paying_capacity_data.csv'))
paying_capacity.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
paying_capacity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

При просмотре первых 5 и 15-20 (для чуть более широкой картины) строчек датасета уже можно заметить ряд ошибок в данных. Сразу бросается в глаза колонка **days_employed** с отрицательными значениями, также можно заметить очень большое число на пятой строчке. Отрицательные значения скорее всего появились вследствие технической ошибки при сборе данных - записывались с дефисом, значение пятой строчки данного столбца скорее записано не в количестве дней, а в иных единицах измерения (часах).

Также можно отметить разный регистр записи данных в колонке **education**. Скорее всего данные были собраны с разных систем, в каждой из которых был свой стиль записи, либо клиенты вводили значения вручную при подаче заявки, но данный вариант менее вероятен для этой графы.

Некоторые одинаковые цели получения кредита записаны разными формулировками, например "на покупку автомоболя" и "приобретение автомобиля". Тут уже очевидно, что клиенты заполняли данную графу вручную.

Названия колонок переименовывать не нужно, они хорошо отображают информацию, слова разделены между собой нижним подчеркиванием и все записано нижним регистром на английском языке.
Почти во всех колонках отсутствуют пропущенные значения, кроме **days_employed** и **total_income**, скорее всего клиенты не предоставили банку данную информацию. 

### Шаг 2. Предобработка данных <a name=Шаг2> </a>

### Обработка пропусков

Для начала приведем значения в колонке **days_employed** в обрабатываемый вид:
* Переведем отрицательные значения колонки в положительные взятием под модуль.  
* В этой же колонке выявим сверхбольшие значения с помощью условия, что число превышает 29200 (29200 дней = 80 лет - максимально-возможная продолжительность трудоустройства с запасом). Переведем сверхбольшие значения, которые теоретически записаны не в днях, а в часах - в дни, поделив на 24.

In [3]:
paying_capacity['days_employed'] = paying_capacity['days_employed'].abs()
paying_capacity.loc[paying_capacity['days_employed'] > 29200, 'days_employed'] = paying_capacity['days_employed'] / 24

paying_capacity.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Заполним пропуски в колонке **days_employed** на общую медиану, а пропуски в колонке **total_income** на медианы по категориям доходов из колонки **income_type**.

In [5]:
days_employed_avg = paying_capacity['days_employed'].median()
paying_capacity['days_employed'] = paying_capacity['days_employed'].fillna(days_employed_avg)

income_types = paying_capacity['income_type'].unique()
for income_type in income_types:
    median_for_type = paying_capacity.loc[paying_capacity['income_type'] == income_type, 'total_income'].median()
    paying_capacity.loc[paying_capacity['income_type'] == income_type, 'total_income'] = paying_capacity['total_income'].fillna(median_for_type)

paying_capacity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Пропуски были найдены в двух колонках: **days_employed** и **total_income**. Интересно, что количество пропусков в обоих полностью совпадает. Это может сказать нам, что данное количество людей не предоставили банку информацию по этим двум пунктам. 

Пропущенные значения в колонке **days_employed** можно заменить медианным значением по всей колонке в целом, потому что, вероятнее всего, значения колонки не зависят от значений в других колонках.

Ежемесячный доход в колонке **total_income** скорее всего может зависеть от рода деятельности клиентов, поэтому замена пропусков на общую медиану не подойдет в этом случае. Для замены пропусков найдем медиану для каждого отдельного рода занятости и в соответствии с этим условием произведем замену, где это необходимо.

Проверка через метод info показывает, что все пропущенные значения были заполнены. 

### Замена типа данных

In [5]:
paying_capacity['days_employed'] = paying_capacity['days_employed'].astype(int)
paying_capacity['total_income'] = paying_capacity['total_income'].astype(int)
paying_capacity.info()
paying_capacity.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Снова обратимся к колонкам **days_employed** и **total_income** и заменим их тип данных с вещественного на целочисленный - это поможет лучше воспринимать числа без длинных "хвостиков". Так как округление значений по правилам математики не критично для наших значений, воспользовались методом **astype**.

### Обработка дубликатов

Прежде чем начать поиск дубликатов приведем все значения колонки **education** к нижнему регистру.

In [6]:
paying_capacity['education'] = paying_capacity['education'].str.lower()

Проверим какие уникальные значения есть в колонке **children**.

In [7]:
paying_capacity['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Проверка записей в графе с количеством детей показала, что и здесь встречаются отрицательные и сверхбольшие значения. Причиной может быть техническая ошибка: -1 записалось вместо просто 1, а к 2 приписался 0.

In [8]:
paying_capacity['children'] = paying_capacity['children'].abs()
paying_capacity.loc[paying_capacity['children'] == 20, 'children'] = 2
paying_capacity['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [9]:
duplicates_amount = paying_capacity.duplicated().sum() / len(paying_capacity)
print('Процент дублирующихся строк: {:.2%}'.format(duplicates_amount))

Процент дублирующихся строк: 0.33%


Процент задублированных строк довольно мал по отношению ко всему датасету, поэтому все повторяющиеся строки можно удалить.

In [10]:
paying_capacity = paying_capacity.drop_duplicates().reset_index(drop=True)
paying_capacity.duplicated().sum()

0

### Вывод

Для проверки датасета на наличие дубликатов, привели значения колонки **education** к нижнему регистру, так как метод **duplicated** воспринимает слова, записанные разным регистром, как разные.

Далее находим процент задублированных строк, чтобы оценить и определить дальнейшие действия. Так как это значение оказалось меньше половины процента, удаляем дубликаты - в этом случае это не повредит общим данным и не скажется на результате.

Возможно, дубликаты появились из-за ошибки в системе сбора данных и часть собралась по нескольку раз. Также возможной причиной могут быть реальные повторные обращения клиентов в банк с предоставлением одинаковых документов, подтверждающих трудовой стаж и ежемесячный доход.

### Лемматизация

Колонка с целями получения кредита содержит записи, записанные по-разному, разными словами. Для того, чтобы в дальнейшем работать с этими значениями, лемматизируем колонку и создадим новую лемматизированную.

In [11]:
from pymystem3 import Mystem
m = Mystem()

def lemm(purpose):
    lemmas = ' '.join(m.lemmatize(purpose))
    return lemmas
 
paying_capacity['purpose_lemm'] = paying_capacity['purpose'].apply(lemm)

paying_capacity.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemm
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,дополнительный образование \n
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,сыграть свадьба \n
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,покупка жилье \n
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операция с жилье \n
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование \n
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,на проведение свадьба \n
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,покупка жилье для семья \n


### Вывод

Для лемматизации столбца с целями получения кредита была создана специальная функция. Далее применили эту функцию к столбцу с целями получения кредита и создали новую колонку с лемматизированными значениями для каждой строки. 

### Категоризация данных

С помощью метода **value_counts**, примененного к колонке **purpose_lemm**, посмотрим, сколько у нас получилось различных лемм с целями получения кредита и число их упоминаний. Из списка отчетливо видно, что есть всего 4 основные категории: автомобиль, недвижимость/жилье, свадьба и образование.

In [12]:
paying_capacity['purpose_lemm'].value_counts()

автомобиль \n                                    972
свадьба \n                                       791
на   проведение   свадьба \n                     768
сыграть   свадьба \n                             765
операция   с   недвижимость \n                   675
покупка   коммерческий   недвижимость \n         661
операция   с   жилье \n                          652
покупка   жилье   для   сдача \n                 651
операция   с   коммерческий   недвижимость \n    650
покупка   жилье \n                               646
жилье \n                                         646
покупка   жилье   для   семья \n                 638
строительство   собственный   недвижимость \n    635
недвижимость \n                                  633
операция   со   свой   недвижимость \n           627
строительство   жилой   недвижимость \n          624
покупка   недвижимость \n                        621
покупка   свой   жилье \n                        620
строительство   недвижимость \n               

Напишем функцию, которая запишет все цели одним словом для каждой категории, чтобы потом можно было данные группировать по категориям. 

In [13]:
def purpose_type(purpose):
    if 'автомобиль' in purpose:
        return 'автомобиль'
    elif 'образование' in purpose:
        return 'образование'
    elif 'свадьба' in purpose:
        return 'свадьба'
    elif 'недвижимость' or 'жилье' in purpose:
        return 'недвижимость'
    

paying_capacity['purpose_type'] = paying_capacity['purpose_lemm'].apply(purpose_type)
paying_capacity.drop(['purpose', 'purpose_lemm'], axis='columns', inplace=True)
paying_capacity.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose_type
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость


Сделали отдельную колонку с целью получения кредита, где оставили только 1 ключевое слово-категорию, остальные 2 прошлые колонки с целями удалили.

Осуществим также распределение по категориям в зависимости от размера ежемесячного дохода клиента, категории запишем в новый столбец.

In [14]:
def income_amount_type(total_income):
    if total_income < 60000:
        return 'доход до 60000р'
    elif 60000 <= total_income < 150000:
        return 'доход от 60000р до 150000р'
    elif 150000 <= total_income < 500000:
        return 'доход от 150000р до 500000р'
    elif 500000 <= total_income < 1000000:
        return 'доход от 500000р до 1000000р'
    return 'доход более 1000000р'
    
paying_capacity['income_amount_type'] = paying_capacity['total_income'].apply(income_amount_type)

### Вывод

Разбили данные на 4 четкие категории по целям получения кредита, категории выбирались логическим путем, анализируя данные после лемматизации данной колонки.

Добавили новую колонку с категориями клиентов в зависимости от уровня ежемесячного дохода.

### Шаг 3. Ответы на вопросы <a name=Шаг3> </a>

#### 3.1 Есть ли зависимость между наличием детей и возвратом кредита в срок? <a name=Шаг31></a>

In [15]:
paying_capacity['debt_1'] = paying_capacity['debt']
children_pivot = paying_capacity.pivot_table(index='debt_1', columns='children', values='debt', aggfunc='count', margins=True)

debt_children = paying_capacity.loc[(paying_capacity['debt'] == 1) & (paying_capacity['children'] != 0), 'debt'].count()
no_debt_children = paying_capacity.loc[(paying_capacity['debt'] == 0) & (paying_capacity['children'] != 0), 'debt'].count()
children_client = paying_capacity.loc[paying_capacity['children'] != 0, 'children'].count()

no_children_part = debt_children / paying_capacity.loc[(paying_capacity['debt'] == 0) & (paying_capacity['children'] == 0), 'debt'].count()
children_part = debt_children / no_debt_children

print('Количество клиентов с детьми:', children_client)
print()
print('Доля клиентов с задолженностью без детей по отношению к количеству клиентов')
print('без задолженности и без детей: {:.2%}'.format(no_children_part))
print()
print('Доля клиентов с задолженностью и детьми по отношению к количеству клиентов')
print('без задолженности с детьми: {:.2%}'.format(children_part))
children_pivot

Количество клиентов с детьми: 7363

Доля клиентов с задолженностью без детей по отношению к количеству клиентов
без задолженности и без детей: 5.20%

Доля клиентов с задолженностью и детьми по отношению к количеству клиентов
без задолженности с детьми: 10.14%


children,0,1,2,3,4,5,All
debt_1,,,,,,,
0,13028.0,4410.0,1926.0,303.0,37.0,9.0,19713
1,1063.0,445.0,202.0,27.0,4.0,NaN,1741
All,14091.0,4855.0,2128.0,330.0,41.0,9.0,21454


In [16]:
new_children_pivot = children_pivot / children_pivot.iloc[-1]
new_children_pivot.applymap('{:.2%}'.format)

children,0,1,2,3,4,5,All
debt_1,,,,,,,
0,92.46%,90.83%,90.51%,91.82%,90.24%,100.00%,91.88%
1,7.54%,9.17%,9.49%,8.18%,9.76%,nan%,8.12%
All,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


### Вывод

Для ответа на вопрос влияет ли наличие детей на возврат кредита в срок сравним данные по двум категориям клиентов: с детьми и без.

Во-первых, в целом количество клиентов с детьми почти в 2 раза меньше, чем клиентов без детей. Значит, что люди, у которых еще нет детей, больше склонны к тому, чтобы взять кредит. Скорее всего у таких людей на данный момент другие приоритеты и цели, на которые они берут заемные средства и нет дополнительного страха за семью, в случае, если они по каким-то причинам не смогут выплатить долг.

Во-вторых, если сравнить долю клиентов с задолженностью по отношению к клиентам без задолженности в рамках категорий с детьми и без, то получится, что доля должников с детьми в 2 раза больше должников без детей. **Значит, зависимость все таки наблюдается не в пользу клиентов с детьми.**

Также, из сводной таблицы видно, что с увеличением количества детей, количество клиентов уменьшается. Скорее всего это обусловлено тем, что в целом семей с бОльшим количеством детей меньше.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок? <a name=Шаг32></a>

In [17]:
family_status_pivot = paying_capacity.pivot_table(index='debt_1', columns='family_status', values='debt', aggfunc='count', margins=True)
family_status_pivot

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем,All
debt_1,,,,,,
0,2536,1110,896,3763,11408,19713
1,274,85,63,388,931,1741
All,2810,1195,959,4151,12339,21454


In [18]:
family_new = family_status_pivot / family_status_pivot.iloc[-1]
family_new.applymap('{:.2%}'.format)

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем,All
debt_1,,,,,,
0,90.25%,92.89%,93.43%,90.65%,92.45%,91.88%
1,9.75%,7.11%,6.57%,9.35%,7.55%,8.12%
All,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


### Вывод

Данные сводной таблицы с процентами говорят нам, что самые прилежные плательщики по кредитам - вдовцы и вдовы. На втором месте с почти одинаковыми позициями идут клиенты, состоящие в браке и те, кто там был и уже развелся. Далее же также с почти одинаковыми позициями идут клиенты, не состоящие в браке и находящиеся в гражданском браке.

Такие результаты могут быть связаны с тем, что клиенты в браке, в разводе, либо овдовевшие клиенты имеют более стабильную жизнь и источники дохода, они более системные и организованные, в отличие от клиентов не состоящих в браке, либо находящихся в гражданском браке.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок? <a name=Шаг33></a>

In [19]:
total_income_pivot = paying_capacity.pivot_table(index='debt_1', columns='income_amount_type', values='debt', aggfunc='count', margins=True)
total_income_pivot

income_amount_type,доход более 1000000р,доход до 60000р,доход от 150000р до 500000р,доход от 500000р до 1000000р,доход от 60000р до 150000р,All
debt_1,,,,,,
0,23,757,8860,185,9888,19713
1,2,49,749,12,929,1741
All,25,806,9609,197,10817,21454


In [20]:
income_new = total_income_pivot / total_income_pivot.iloc[-1]
income_new.applymap('{:.2%}'.format)

income_amount_type,доход более 1000000р,доход до 60000р,доход от 150000р до 500000р,доход от 500000р до 1000000р,доход от 60000р до 150000р,All
debt_1,,,,,,
0,92.00%,93.92%,92.21%,93.91%,91.41%,91.88%
1,8.00%,6.08%,7.79%,6.09%,8.59%,8.12%
All,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%


### Вывод

По сводной таблице, разбитой на категории по уровню дохода, нет какой-то четко прослеживающейся тенденции.

Лучшие плательщики - клиенты из самой бедной категории с доходом **до 60000р**, это можно объяснить тем, что люди не хотят рисковать лишними штрафами за просрочку выплат и более ответственно к этому подходят.

Клиенты из следующей по уровню дохода категории **от 60000р до 150000р** наоборот показали себя хуже всего. Возможно люди данной категории уже могут позволить себе большее, но еще не всё, берут кредит на крупные цели и вследствие этого имеют трудности с выплатой.

Две следующие категории с доходом **от 150000р до 500000р** и **от 500000р до 1000000р** могут показать тенденцию, что все таки людям с бОльшей доходностью легче дается погашение кредита в срок, но категория людей с доходом **от 1000000р** показывает снова обратную тенденцию, что не дает нам сделать подобный вывод.

#### 3.4 Как разные цели кредита влияют на его возврат в срок? <a name=Шаг34></a>

In [21]:
family_status_pivot = paying_capacity.pivot_table(index='debt_1', columns='purpose_type', values='debt', aggfunc='count', margins=True)
family_status_pivot

purpose_type,автомобиль,недвижимость,образование,свадьба,All
debt_1,,,,,
0,3903,10029,3643,2138,19713
1,403,782,370,186,1741
All,4306,10811,4013,2324,21454


In [22]:
new_family_status = family_status_pivot / family_status_pivot.iloc[-1]
new_family_status.applymap('{:.2%}'.format)

purpose_type,автомобиль,недвижимость,образование,свадьба,All
debt_1,,,,,
0,90.64%,92.77%,90.78%,92.00%,91.88%
1,9.36%,7.23%,9.22%,8.00%,8.12%
All,100.00%,100.00%,100.00%,100.00%,100.00%


### Вывод

Самая популярная цель получения кредита - кредит на недвижимость, его взяли почти половина всех клиентов банка. Недвижимость, особенно жилая, это жизненная необходимость. Можно отметить, что именно в этой категории лучше всего с оплатой в срок, так как люди не хотят рисковать собственным жильем.

На втором месте по успешности выплаты кредита в срок - свадьбы. Скорее всего это обусловленно мЕньшими суммами займов, по сравнению с другими категориями, и такие суммы легче выплатить (особенно если гости дарили деньги в качестве свадебного подарка).

Далее идут кредиты на образование, они более популярны, чем кредиты на свадьбу, но выплачивают в срок их хуже. Возможно, такие кредиты чаще берут родители для платного образования своих детей в семьях, чей достаток и собственные накопления не позволяют сразу оплатить учебу. Скорее всего подобные финансовые проблемы и являются причиной задержек с выплатой.

Чуть хуже, но почти одинаковая ситуация с выплатами по кредитам на автомобиль. Количество клиентов также почти одинаковое с теми клиентами, кто взял кредит на образование. Автомобиль не такая первостепенная потребность как, например, жилье, но зачастую и не дешевая. Люди скорее всего в меньшей степени боятся последствий за несвоевременную уплату долга по такому кредиту.

### Шаг 4. Общий вывод <a name=Шаг4></a>

Данные датасета содержали в себе ряд моментов, требующих предварительной обработки: отрицательные значения, сверхбольшие значения, выбивающиеся из общего ряда, пропуски, разный регистр записи данных, а также ответы клиентов, записанные в произвольной форме.

Все данные были приведены в удобный для дальнейшей обработки и анализа вид.

Сводные таблицы хорошо показывают нам, как на оплату кредита в срок влияют разные категории клиентов и цели, на которые они получили кредит. Так, например, можно сделать вывод, что самые надежные плательщики это вдовы/вдовцы без детей, покупающие недвижимость и с доходом либо до 60000р, либо от 500000р до 1000000р.

Самая же неблагонадежная группа клиентов - люди, не состоящие в официальном браке, с детьми, доходом от 60000р до 150000р и желающие приобрести автомобиль. 